<a href="https://colab.research.google.com/github/cjkipu/ML-Group-Final-Task/blob/main/Language_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing of necessary Libraries**

In [35]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense

In [1]:
from google.colab import files  
uploaded = files.upload()

Saving lang_detect.csv to lang_detect (3).csv


In [4]:
df = pd.read_csv('lang_detect (3).csv', sep='\t', encoding='utf8', index_col=0, names=['lang','text'])

In [6]:
df.head()

,lang,text
1,cmn,我們試試看！
2,cmn,我该去睡觉了。
3,cmn,你在干什麼啊？
4,cmn,這是什麼啊？
5,cmn,今天是６月１８号，也是Muiriel的生日！


**Data Preprocessing**

In [8]:
length_cond = [True if 20<=len(s)<=200 else False for s in df['text']]
df = df[length_cond]

In [10]:
language = ['eng', 'deu', 'fra', 'ita', 'spa', 'por']
df = df[df['lang'].isin(language)]

In [11]:
#Selecting 50000 rows for each language
data_trim = pd.DataFrame(columns=['lang','text'])

for l in language:
    lang_trim = df[df['lang'] ==l].sample(50000, random_state = 100)
    data_trim = data_trim.append(lang_trim)

**Data Set Splitting into Train and Test Sets**

In [12]:
data_shuffle = data_trim.sample(frac=1)

train = data_shuffle[0:210000]
valid = data_shuffle[210000:270000]
test = data_shuffle[270000:300000]

**Feature Creation**

In [13]:
def get_trigrams(corpus,n_feat=200):
    vectorizer = CountVectorizer(analyzer='char',
                            ngram_range=(3, 3)
                            ,max_features=n_feat)
    
    X = vectorizer.fit_transform(corpus)
    feature_names = vectorizer.get_feature_names()
    return feature_names

In [15]:
features = {}
features_set = set()
for l in language:
    corpus = train[train.lang==l]['text']
    trigrams = get_trigrams(corpus)
    features[l] = trigrams 
    features_set.update(trigrams)
vocab = dict()
for i,f in enumerate(features_set):
    vocab[f]=i

In [16]:
vectorizer = CountVectorizer(analyzer='char',
                             ngram_range=(3, 3),
                            vocabulary=vocab)
corpus = train['text']   
X = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names()
train_feat = pd.DataFrame(data=X.toarray(),columns=feature_names)

In [17]:
train_min = train_feat.min()
train_max = train_feat.max()
train_feat = (train_feat - train_min)/(train_max-train_min)
train_feat['lang'] = list(train['lang'])

In [19]:
corpus = valid['text']   
X = vectorizer.fit_transform(corpus)

valid_feat = pd.DataFrame(data=X.toarray(),columns=feature_names)
valid_feat = (valid_feat - train_min)/(train_max-train_min)
valid_feat['lang'] = list(valid['lang'])

corpus = test['text']   
X = vectorizer.fit_transform(corpus)

test_feat = pd.DataFrame(data=X.toarray(),columns=feature_names)
test_feat = (test_feat - train_min)/(train_max-train_min)
test_feat['lang'] = list(test['lang'])

**Label Encoder**

In [21]:
encoder = LabelEncoder()
encoder.fit(['eng', 'deu', 'fra', 'ita', 'spa', 'por'])

def encode(y):
    y_encoded = encoder.transform(y)
    y_dummy = np_utils.to_categorical(y_encoded)
    return y_dummy

In [22]:
x = train_feat.drop('lang',axis=1)
y = encode(train_feat['lang'])

**Model Creation and Training**

In [29]:
epochs = 8

model = Sequential()
model.add(Dense(500, input_dim=669, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(6, activation='softmax'))

#Metrics we are tracking is accuracy of the model in detecting the language
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x, y, epochs = epochs, batch_size=100)

Epoch 1/8
2100/2100 [==============================] - 30s 14ms/step - loss: 0.0730 - accuracy: 0.9749
Epoch 2/8
2100/2100 [==============================] - 29s 14ms/step - loss: 0.0334 - accuracy: 0.9877
Epoch 3/8
2100/2100 [==============================] - 29s 14ms/step - loss: 0.0195 - accuracy: 0.9931
Epoch 4/8
2100/2100 [==============================] - 29s 14ms/step - loss: 0.0118 - accuracy: 0.9959
Epoch 5/8
2100/2100 [==============================] - 29s 14ms/step - loss: 0.0079 - accuracy: 0.9972
Epoch 6/8
2100/2100 [==============================] - 29s 14ms/step - loss: 0.0064 - accuracy: 0.9979
Epoch 7/8
2100/2100 [==============================] - 30s 14ms/step - loss: 0.0048 - accuracy: 0.9984
Epoch 8/8
2100/2100 [==============================] - 30s 14ms/step - loss: 0.0037 - accuracy: 0.9988
